In [1]:
import pandas as pd

df = pd.read_csv('./data/train.csv')

In [2]:
df['newsSubTitle'] = df['newsSubTitle'].fillna("부제목없음")

In [3]:
drop = ["Unnamed: 0.1",	"Unnamed: 0","processSentencenum"]
df = df.drop(columns=drop)

In [4]:
df['newsSubcategory'] = df['newsSubcategory'].fillna("부카테고리없음")

In [5]:
df.isnull().sum()

newsID             0
newsCategory       0
newsSubcategory    0
newsTitle          0
newsSubTitle       0
newsContent        0
partNum            0
useType            0
processType        0
processPattern     0
processLevel       0
sentenceCount      0
sentenceInfo       0
dtype: int64

validation

In [11]:
import pandas as pd

df_validation = pd.read_csv('./data/validation.csv')

In [12]:
df_validation.isnull().sum()

Unnamed: 0.1              0
Unnamed: 0                0
newsID                    0
newsCategory              0
newsSubcategory         210
newsTitle                 0
newsSubTitle          45703
newsContent               0
partNum                   0
useType                   0
processType               0
processPattern            0
processLevel              0
sentenceCount             0
sentenceInfo              0
processSentencenum    34418
dtype: int64

In [13]:
drop = ["Unnamed: 0.1",	"Unnamed: 0","processSentencenum"]
df_validation = df_validation.drop(columns=drop)
df_validation['newsSubTitle'] = df_validation['newsSubTitle'].fillna("부제목없음")
df_validation['newsSubcategory'] = df_validation['newsSubcategory'].fillna("부카테고리없음")

In [ ]:
# 'train.csv'로 저장
df.to_csv('./data/train.csv', index=False)

# 'validation.csv'로 저장
df_validation.to_csv('./validation.csv', index=False)

RoBERTa

In [3]:

import os
import tqdm
import pandas as pd
import torch
import datasets

from glob import glob
from tqdm import tqdm

from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

In [4]:
MODEL_NAME = "klue/roberta-large" # "klue/bert-base", "klue/bert-large", "klue/roberta-base"
batch_size = 64
learning_rate = 5e-5
num_epochs = 1

In [5]:
raw_train = load_dataset('csv', data_files='../data/train_1.csv')
raw_test = load_dataset('csv', data_files='../data/validation_1.csv')

In [6]:
train, valid= raw_train['train'].train_test_split(test_size=0.1).values()
review_dataset = datasets.DatasetDict({'train': train, 'valid': valid, 'test': raw_test['train']})
review_dataset

DatasetDict({
    train: Dataset({
        features: ['newsID', 'newsCategory', 'newsSubcategory', 'newsTitle', 'newsSubTitle', 'newsContent', 'partNum', 'useType', 'processType', 'processPattern', 'processLevel', 'sentenceCount', 'sentenceInfo'],
        num_rows: 202948
    })
    valid: Dataset({
        features: ['newsID', 'newsCategory', 'newsSubcategory', 'newsTitle', 'newsSubTitle', 'newsContent', 'partNum', 'useType', 'processType', 'processPattern', 'processLevel', 'sentenceCount', 'sentenceInfo'],
        num_rows: 22550
    })
    test: Dataset({
        features: ['newsID', 'newsCategory', 'newsSubcategory', 'newsTitle', 'newsSubTitle', 'newsContent', 'partNum', 'useType', 'processType', 'processPattern', 'processLevel', 'sentenceCount', 'sentenceInfo'],
        num_rows: 69280
    })
})

In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(tokenizer.tokenize(train['newsTitle'][0]))
print(tokenizer.tokenize(train['newsSubTitle'][0]))
print(tokenizer.tokenize(train['newsSubcategory'][0]))
print(tokenizer.tokenize(train['newsContent'][0]))

def tokenize_function(example):
    # 여러 컬럼을 각각 토큰화
    title_tokens = tokenizer(example["newsTitle"], truncation=True)
    subtitle_tokens = tokenizer(example["newsSubTitle"], truncation=True)
    newsContent = tokenizer(example['newsContent'], truncation=True)
    newsSubcategory = tokenizer(example['newsSubcategory'], truncation=True)
    return {**title_tokens, **subtitle_tokens, **newsContent, **newsSubcategory}


tokenized_datasets = review_dataset.map(tokenize_function, batched=True)


['미국', '노스', '##캐', '##롤', '##라이나', '##서', '또', '총격', '…', '3', '##명', '사망', ',', '4', '##명', '부상']
['부제', '##목', '##없', '##음']
['미국', '.', '캐나다']
['미국', '노스', '##캐', '##롤', '##라이나', '##주', '##에서', '총격', '사건', '##이', '발생', '##해', '3', '##명', '##이', '숨지', '##고', '4', '##명', '##이', '부상', '##했', '##다고', 'CNN', '##이', '3', '##일', '(', '현지', '##시간', ')', '보도', '##했', '##다', '.', '보도', '##에', '따르', '##면', '경찰', '##은', '이날', '오전', '노스', '##캐', '##롤', '##라이나', '##주', '윌', '##밍', '##턴', '##의', '한', '가정', '##집', '##에서', '총격', '##으로', '3', '##명', '##이', '사망', '##하고', '4', '##명', '##이', '다쳤', '##다고', '밝혔', '##다', '.', '경찰', '##은', '자정', '직후', '신고', '##를', '받', '##고', '출동', '##했', '##다', '.', '이', '곳', '##에서', '##는', '파티', '##가', '벌어지', '##고', '있', '##던', '것', '##으로', '전해졌', '##다', '.', '경찰', '##은', '보도', '##자료', '##에서', '이번', '사건', '##을', '두고', '[UNK]', '"', '총격', '##전', '(', 'g', '##un', '##f', '##ight', ')', '[UNK]', '"', '이라', '##고', '표현', '##했', '##으나', '구체', '##적인', '내용', '##은', '밝히', '##지', 

Map:   0%|          | 0/202948 [00:00<?, ? examples/s]

Map:   0%|          | 0/22550 [00:00<?, ? examples/s]

Map:   0%|          | 0/69280 [00:00<?, ? examples/s]

{'labels': tensor(1), 'input_ids': tensor([    0,  3634,  2267,  2616,  2200, 12865,  5028,     2]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1])}


In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['newsID', 'newsCategory', 'newsSubcategory', 'newsTitle', 'newsSubTitle', 'newsContent', 'partNum', 'useType', 'processType', 'processPattern', 'processLevel', 'sentenceCount', 'sentenceInfo', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 202948
    })
    valid: Dataset({
        features: ['newsID', 'newsCategory', 'newsSubcategory', 'newsTitle', 'newsSubTitle', 'newsContent', 'partNum', 'useType', 'processType', 'processPattern', 'processLevel', 'sentenceCount', 'sentenceInfo', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 22550
    })
    test: Dataset({
        features: ['newsID', 'newsCategory', 'newsSubcategory', 'newsTitle', 'newsSubTitle', 'newsContent', 'partNum', 'useType', 'processType', 'processPattern', 'processLevel', 'sentenceCount', 'sentenceInfo', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 69280
    })
})

In [12]:
tokenized_datasets = tokenized_datasets.remove_columns(['newsID', 'newsCategory', 'newsSubcategory', 'newsTitle', 'newsSubTitle', 'newsContent', 'partNum','processType', 'processPattern', 'processLevel', 'sentenceCount', 'sentenceInfo' ])
tokenized_datasets['train'] = tokenized_datasets['train'].rename_column("useType", "labels")
tokenized_datasets['valid'] = tokenized_datasets['valid'].rename_column("useType", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [13]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=batch_size, collate_fn=data_collator)
valid_dataloader = DataLoader(tokenized_datasets["valid"], shuffle=True, batch_size=batch_size, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets["test"], shuffle=False, batch_size=batch_size, collate_fn=data_collator)

In [14]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([64]),
 'input_ids': torch.Size([64, 11]),
 'token_type_ids': torch.Size([64, 11]),
 'attention_mask': torch.Size([64, 11])}

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=6) # 편의상 6으로 설정

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from transformers import get_scheduler, AdamW
optimizer = AdamW(model.parameters(), lr=learning_rate)

num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

/Users/rirakang/virtual/envev/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


3172


In [17]:
import torch

# MPS 장치가 가능하면 사용, 아니면 CPU 사용
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"사용 중인 장치: {device}")

# 모델을 해당 장치로 이동
model.to(device)


사용 중인 장치: mps


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=Tru

In [1]:
import torch
import time
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 장치 설정
device_cpu = torch.device("cpu")
device_mps = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# 모델과 토크나이저 로드
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 더미 데이터 준비
text = ["This is a test sentence for performance evaluation."] * 1000
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# CPU에서 테스트
model.to(device_cpu)
inputs = {k: v.to(device_cpu) for k, v in inputs.items()}
start_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
cpu_time = time.time() - start_time

# MPS에서 테스트
model.to(device_mps)
inputs = {k: v.to(device_mps) for k, v in inputs.items()}
start_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
mps_time = time.time() - start_time

print(f"CPU inference time: {cpu_time:.4f} seconds")
print(f"MPS inference time: {mps_time:.4f} seconds")


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

CPU inference time: 2.4115 seconds
MPS inference time: 2.1613 seconds


In [18]:
import torch

# MPS 사용 여부 확인
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# 모델과 데이터 로드
model.to(device)

# Training Loop
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # 모델과 토크나이저 저장
    model.save_pretrained(f"./result/{MODEL_NAME}/{epoch}")
    tokenizer.save_pretrained(f"./result/{MODEL_NAME}/{epoch}")


Using device: mps


  0%|          | 0/3172 [00:00<?, ?it/s]

In [34]:
!pip install torchmetrics


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.4/891.4 kB 8.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [19]:
from torchmetrics import Accuracy
import torch

# 이진 분류용 Accuracy 메트릭 초기화
accuracy = Accuracy(task="binary").to(device)

prediction_list_valid = []
target_list_valid = []

model.eval()
for batch in valid_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).cpu()
    targets = batch['labels'].cpu()

    # 리스트에 예측값과 타겟값 추가
    prediction_list_valid.extend(predictions)
    target_list_valid.extend(targets)

# 예측값과 타겟값을 텐서로 변환
preds_tensor = torch.tensor(prediction_list_valid)
targets_tensor = torch.tensor(target_list_valid)

# Accuracy 계산
valid_acc = accuracy(preds_tensor, targets_tensor).item()
print(f'valid acc: {valid_acc:.4f}')


valid acc: 0.7292
